In [1]:
import os
from snowflake.snowpark import Session
from snowflake.snowpark.version import VERSION
from snowflake.snowpark.types import StructType, StructField, DoubleType, StringType
import snowflake.snowpark.functions as F

In [2]:
connection_parameters = {
    "account": "ug94937.us-east4.gcp",
    "user": "ADITYASINGH",
    "password": os.environ.get('SF_Password'),
    "role": "ADITYASINGH",  # optional
    "warehouse": "FOSFOR_INSIGHT_WH",  # optional
#     "authenticator": "externalbrowser", # optional
    "database": "FIRST_DB",  # optional
    "schema": "PUBLIC",  # optional
} 

In [3]:
# Make a Snowpark Connection

################################################################################################################
#  You can also use the SnowSQL Client to configure your connection params:
#  https://docs.snowflake.com/en/user-guide/snowsql-install-config.html
#
#  >>> from snowflake.ml.utils import connection_params
#  >>> session = Session.builder.configs(connection_params.SnowflakeLoginOptions()
#  >>> ).create()   
#
#  NOTE: If you have named connection params then specify the connection name
#  Example:
#  
#  >>> session = Session.builder.configs(
#  >>> connection_params.SnowflakeLoginOptions(connection_name='connections.snowml')
#  >>> ).create()
#
#################################################################################################################

# Edit the connection.json before creating the session object below
# Create Snowflake Session object
# connection_parameters = json.load(open('connection.json'))
session = Session.builder.configs(connection_parameters).create()
session.sql_simplifier_enabled = True

snowflake_environment = session.sql('SELECT current_user(), current_version()').collect()
snowpark_version = VERSION

# Current Environment Details
print('\nConnection Established with the following parameters:')
print('User                        : {}'.format(snowflake_environment[0][0]))
print('Role                        : {}'.format(session.get_current_role()))
print('Database                    : {}'.format(session.get_current_database()))
print('Schema                      : {}'.format(session.get_current_schema()))
print('Warehouse                   : {}'.format(session.get_current_warehouse()))
print('Snowflake version           : {}'.format(snowflake_environment[0][1]))
print('Snowpark for Python version : {}.{}.{}'.format(snowpark_version[0],snowpark_version[1],snowpark_version[2]))


Connection Established with the following parameters:
User                        : ADITYASINGH
Role                        : "ADITYASINGH"
Database                    : "FIRST_DB"
Schema                      : "PUBLIC"
Warehouse                   : "FOSFOR_INSIGHT_WH"
Snowflake version           : 8.20.10
Snowpark for Python version : 1.17.0


In [14]:
from snowflake.ml.modeling.pipeline import Pipeline
from snowflake.ml.modeling.preprocessing import MinMaxScaler, OrdinalEncoder
from snowflake.ml.modeling.metrics import mean_squared_error, mean_absolute_error, r2_score
from snowflake.ml.modeling.xgboost import XGBClassifier
from snowflake.snowpark import Session, FileOperation

In [13]:
# data = session.table('EMPLOYEE')
df_train, df_test = session.table('EMPLOYEE').drop('ROW').random_split(weights=[0.9, 0.1], seed=0)

In [15]:
df_train.show()

-------------------------------------------------------------------------------------------------------------------------------------------
|"EDUCATION"  |"JOININGYEAR"  |"CITY"     |"PAYMENTTIER"  |"AGE"  |"GENDER"  |"EVERBENCHED"  |"EXPERIENCEINCURRENTDOMAIN"  |"LEAVEORNOT"  |
-------------------------------------------------------------------------------------------------------------------------------------------
|Bachelors    |2017           |Bangalore  |3              |34     |Male      |No             |0                            |0             |
|Bachelors    |2013           |Pune       |1              |28     |Female    |No             |3                            |1             |
|Bachelors    |2014           |New Delhi  |3              |38     |Female    |No             |2                            |0             |
|Masters      |2016           |Bangalore  |3              |27     |Male      |No             |5                            |1             |
|Masters      |2017 

In [7]:
cols = df_train.columns
print(cols)

['EDUCATION', 'JOININGYEAR', 'CITY', 'PAYMENTTIER', 'AGE', 'GENDER', 'EVERBENCHED', 'EXPERIENCEINCURRENTDOMAIN', 'LEAVEORNOT']


In [8]:
data_schema = session.sql("DESCRIBE TABLE EMPLOYEE").collect()

In [9]:
categorical_types = ['VARCHAR','CHAR','STRING','TEXT','BOOL']
categorical_columns = []
for row in data_schema:
    for typ in categorical_types:
        if typ in row['type']:
            categorical_columns.append(row['name'])
            break

In [11]:
numerical_columns = list(set(cols) - set(categorical_columns))

In [12]:
print(numerical_columns)
print(categorical_columns)

['JOININGYEAR', 'PAYMENTTIER', 'LEAVEORNOT', 'AGE', 'EXPERIENCEINCURRENTDOMAIN']
['EDUCATION', 'CITY', 'GENDER', 'EVERBENCHED']


In [18]:
categorical_columns_oe = list(map(lambda a: a+'_OE', categorical_columns))

In [19]:
print(categorical_columns_oe)
print(categorical_columns)

['EDUCATION_OE', 'CITY_OE', 'GENDER_OE', 'EVERBENCHED_OE']
['EDUCATION', 'CITY', 'GENDER', 'EVERBENCHED']


In [21]:
# Define a pipeline that does the preprocessing and training of 
# a XGBRegressor model
pipe = Pipeline(steps=[
          ("ord", OrdinalEncoder(input_cols=categorical_columns, output_cols=categorical_columns_oe)),
          ("scaler", MinMaxScaler(input_cols=numerical_columns, output_cols=numerical_columns)),
          ("regressor", XGBClassifier(input_cols=categorical_columns_oe+numerical_columns
                                      , label_cols=["LEAVEORNOT"]
                                      , output_cols=['PREDICTION'], n_jobs=-1))
         ]
       )
 
# Fit the pipeline
xgb_model = pipe.fit(df_train)
 
# Test the model
df_test_pred = xgb_model.predict(df_test)
mse = mean_squared_error(df=df_test_pred, y_true_col_names="PRICE", y_pred_col_names="PREDICTION")
mae = mean_absolute_error(df=df_test_pred, y_true_col_names="PRICE", y_pred_col_names="PREDICTION")
r2 = r2_score(df=df_test_pred, y_true_col_name="PRICE", y_pred_col_name="PREDICTION")
print(f'MSE: {mse}')
print(f'MAE: {mae}')
print(f'R2: {r2}')

SnowparkSQLException: (1300) (1304): 01b4a0e3-0000-655a-0000-576d06b8207a: 002028 (42601): SQL compilation error:
ambiguous column name 'LEAVEORNOT'

In [6]:
def encoding(df, target_column):
    """
    Checking whether encoding required in target and feature datasets.
    If required, then encoding them with label and one hot encoding.
    :param:
    df: input dataframe
    target_column: target column
    :returns:
    df_target: target dataframe
    le_target: target label encoder object
    df_feature: feature dataframe
    le_dict_feature: dict of feature label encoder objects
    oh_enc_feature: feature one hot encoder object
    le_column_feature: list of feature label encoder columns
    oh_column_feature: list of feature one hot encoder columns
    """
    df_target = df[[target_column]]
    le_target = None
    # Target column validation and encoding
    if df.dtypes[target_column].name in ['object', 'bool']:
        print(f"target_column is of {df.dtypes[target_column].name} datatype, encoding required.")
        le_target = LabelEncoder()
        df_target[target_column] = pd.DataFrame(le_target.fit_transform(df_target[target_column].astype(str)))
        print(f"Target column label encoded {df_target[target_column]}, object: {le_target}")

    # Feature column validation and encoding
    df_feature = df.drop(target_column, axis=1)
    non_numeric_cols = df_feature.select_dtypes(include=['object', 'bool']).columns.tolist()
    le_dict_feature = {}
    le_column_feature = []
    oh_column_feature = []
    oh_enc_feature = None
    if len(non_numeric_cols) >= 1:
        print(f"{non_numeric_cols} columns are non numeric in feature dataset, encoding required.")
        for col in non_numeric_cols:
            if df_feature[col].nunique() >= 10:
                le_column_feature.append(col)
            else:
                oh_column_feature.append(col)

        print(f"Columns identified to be encoded with label encoder: {le_column_feature}\n"
              f"Columns identified to be encoded with one hot encoder: {oh_column_feature}")

        # columns to be label encoded
        if len(le_column_feature) == 0:
            df_feature = df_feature
        else:
            for col in le_column_feature:
                le_dict_feature[col] = LabelEncoder()
                df_feature[col] = le_dict_feature[col].fit_transform(df_feature[col].astype(str))
                print(f"{col} column label encoded {df_feature[col]}, object: {le_dict_feature[col]}")

        # columns to be one hot encoded
        if len(oh_column_feature) == 0:
            df_feature = df_feature
        else:
            unique_combinations = pd.get_dummies(df_feature[oh_column_feature])
            unique_combinations_list = unique_combinations.columns.tolist()
            oh_enc_feature = OneHotEncoder()
            oh_encoded_array = oh_enc_feature.fit_transform(df_feature[oh_column_feature]).toarray() if len(oh_column_feature) > 1 else oh_enc_feature.fit_transform(df_feature[oh_column_feature]).toarray()
            df_oh_enc = pd.DataFrame(oh_encoded_array, columns=unique_combinations_list)
            df_feature = df_feature.drop(columns=oh_column_feature)
            df_feature = df_feature.join(df_oh_enc)
            print(f"new one hot encoded df: {oh_encoded_array}\n"
                  f"one hot encoder object: {oh_enc_feature}\n")
        print(f"final feature df created: {df_feature}")
    return df_target, le_target, df_feature, le_dict_feature, oh_enc_feature, le_column_feature, oh_column_feature

In [7]:
df_target, le_target, df_feature, le_dict_feature, oh_enc_feature, le_column_feature, oh_column_feature = encoding(data,'LeaveOrNot')

['Education', 'City', 'Gender', 'EverBenched'] columns are non numeric in feature dataset, encoding required.
Columns identified to be encoded with label encoder: []
Columns identified to be encoded with one hot encoder: ['Education', 'City', 'Gender', 'EverBenched']
new one hot encoded df: [[1. 0. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 [1. 0. 0. ... 0. 1. 0.]
 ...
 [0. 1. 0. ... 1. 1. 0.]
 [1. 0. 0. ... 1. 0. 1.]
 [1. 0. 0. ... 1. 0. 1.]]
one hot encoder object: OneHotEncoder()

final feature df created:       JoiningYear  PaymentTier  Age  ExperienceInCurrentDomain  \
0            2017            3   34                          0   
1            2013            1   28                          3   
2            2014            3   38                          2   
3            2016            3   27                          5   
4            2017            3   24                          2   
...           ...          ...  ...                        ...   
4648         2013      

In [8]:
features_pandas = pd.concat([df_feature, df_target], axis=1)
features_pandas.head()

,JoiningYear,PaymentTier,Age,ExperienceInCurrentDomain,Education_Bachelors,Education_Masters,Education_PHD,City_Bangalore,City_New Delhi,City_Pune,Gender_Female,Gender_Male,EverBenched_No,EverBenched_Yes,LeaveOrNot
0,2017,3,34,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0
1,2013,1,28,3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1
2,2014,3,38,2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0
3,2016,3,27,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1
4,2017,3,24,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1


In [9]:
# features_pandas=features_pandas.drop("Unnamed: 0", axis=1)
features_pandas.columns = map(str.upper, features_pandas.columns)
features_pandas.columns = features_pandas.columns.str.replace(' ', '_')
features_pandas.head()

,JOININGYEAR,PAYMENTTIER,AGE,EXPERIENCEINCURRENTDOMAIN,EDUCATION_BACHELORS,EDUCATION_MASTERS,EDUCATION_PHD,CITY_BANGALORE,CITY_NEW_DELHI,CITY_PUNE,GENDER_FEMALE,GENDER_MALE,EVERBENCHED_NO,EVERBENCHED_YES,LEAVEORNOT
0,2017,3,34,0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0
1,2013,1,28,3,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1
2,2014,3,38,2,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0
3,2016,3,27,5,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,1
4,2017,3,24,2,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1


In [10]:
features_df = session.create_dataframe(features_pandas)
features_df.show()

-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"JOININGYEAR"  |"PAYMENTTIER"  |"AGE"  |"EXPERIENCEINCURRENTDOMAIN"  |"EDUCATION_BACHELORS"  |"EDUCATION_MASTERS"  |"EDUCATION_PHD"  |"CITY_BANGALORE"  |"CITY_NEW_DELHI"  |"CITY_PUNE"  |"GENDER_FEMALE"  |"GENDER_MALE"  |"EVERBENCHED_NO"  |"EVERBENCHED_YES"  |"LEAVEORNOT"  |
-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|2017           |3              |34     |0                            |1.0                    |0.0                  |0.0              |1.0               |0.0               

In [11]:
FEATURE_COLUMNS=list(features_df.columns)
FEATURE_COLUMNS.remove('LEAVEORNOT')
# print(FEATURE_COLUMNS)

In [12]:
# Categorize all the features for modeling
# CATEGORICAL_COLUMNS = ["CUT", "COLOR", "CLARITY"]

# CATEGORICAL_COLUMNS_OE = ["FIXED_ACIDITY","VOLATILE_ACIDITY","CITRIC_ACID","RESIDUAL_SUGAR","CHLORIDES","FREE_SULFUR_DIOXIDE","TOTAL_SULFUR_DIOXIDE","DENSITY","PH","SULPHATES","ALCOHOL"] # To name the ordinal encoded columns
# NUMERICAL_COLUMNS = ["CARAT", "DEPTH", "TABLE_PCT", "X", "Y", "Z"]

LABEL_COLUMNS = ["LEAVEORNOT"]
OUTPUT_COLUMNS = ["PREDICTION"]

# input_cols=FEATURE_COLUMNS
# label_cols=LABEL_COLUMNS
# output_cols=OUTPUT_COLUMNS

In [13]:
model = XGBClassifier(
    input_cols=FEATURE_COLUMNS,
    label_cols=LABEL_COLUMNS,
    output_cols=OUTPUT_COLUMNS
)
model.fit(features_df)

# Use the model to make predictions.
predictions = model.predict(features_df)


The version of package 'snowflake-snowpark-python' in the local environment is 1.17.0, which does not fit the criteria for the requirement 'snowflake-snowpark-python'. Your UDF might not work when the package version is different between the server and your local environment.


In [14]:
predictions[OUTPUT_COLUMNS].show()

----------------
|"PREDICTION"  |
----------------
|0             |
|1             |
|0             |
|1             |
|1             |
|0             |
|0             |
|0             |
|0             |
|0             |
----------------

